# Extract tweet entities from annotations db

In [1]:
import os
import json
import shutil
import random
from PIL import Image

import sys
sys.path.insert(0, "../../")
sys.path.insert(0, "../../../hindsight/hindsight_server/")
from annotation_helpers import add_hindsight_frame_path, get_entity_image, visualize_annotations, annotations_to_label_studio, get_entity_image
from annotations_db import HindsightAnnotationsDB

from db import HindsightDB
from utils import make_dir

In [2]:
annotations_db = HindsightAnnotationsDB()
db = HindsightDB()

In [3]:
all_annotations = annotations_db.get_annotations()
all_annotations['x2'] = all_annotations['x'] + all_annotations['w']
all_annotations['y2'] = all_annotations['y'] + all_annotations['h']

In [4]:
all_tweet_annotations = all_annotations.loc[~all_annotations['parent_annotation_id'].isnull()]

In [5]:
check_tweets = set()
for parent_annotations_id in set(all_tweet_annotations['parent_annotation_id']):
    tweet_annotations = all_tweet_annotations.loc[all_tweet_annotations['parent_annotation_id'] == parent_annotations_id]

    if "quoted_tweet" not in set(tweet_annotations['label']):
        multi_labels = tweet_annotations.groupby(['label']).id.count().sort_values(ascending=False).iloc[0]
        if multi_labels > 1:
            check_tweets.add(parent_annotations_id)

# Feed suspisious annotations back to Label Studio

In [21]:
annotations_feedback_dir = "/Users/connorparish/code/hindsight_parsing/data/extracted_entities/tweets/annotation_feedback/"
make_dir(annotations_feedback_dir)

In [11]:
frame_annotations = all_annotations.loc[all_annotations['parent_annotation_id'].isnull()]
frame_annotations = add_hindsight_frame_path(frame_annotations)

/Users/connorparish/code/hindsight_parsing/playground/annotation_feedback/../../annotation_helpers.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations['frame_path'] = annotations['frame_id'].map(frame_id_to_path)


In [38]:
check_frame_anotations = frame_annotations.loc[frame_annotations['id'].isin(check_tweets)]

In [39]:
check_frame_anotations = check_frame_anotations.sample(150)

In [41]:
check_frame_anotations['frame_basename'] = check_frame_anotations['frame_path'].apply(lambda x: os.path.basename(x).split('.')[0])

In [44]:
all_preds = list()
for i, row in check_frame_anotations.iterrows():
    tweet_annotations = all_tweet_annotations.loc[all_tweet_annotations['parent_annotation_id'] == row['id']]
    im = get_entity_image(row)
    im_path = os.path.join(annotations_feedback_dir, f"{row['frame_basename']}-{row['id']}.jpg")
    im.save(im_path)
    preds_d = annotations_to_label_studio(tweet_annotations, im_path, im.width, im.height)
    all_preds.append(preds_d)

with open("preds_2.json", 'w') as outfile:
    json.dump(all_preds, outfile)

/var/folders/c_/2c9vmfhd35bgwc_6h0q80d180000gn/T/ipykernel_18989/3712669943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations['model_name_version'] = annotations['model_name'] + "-" + annotations['model_version']
/var/folders/c_/2c9vmfhd35bgwc_6h0q80d180000gn/T/ipykernel_18989/3712669943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations['model_name_version'] = annotations['model_name'] + "-" + annotations['model_version']
/var/folders/c_/2c9vmfhd35bgwc_6h0q80d180000gn/T/ipykernel